### **Can execute python codes and plot visualizations**

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from dataclasses import dataclass
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64

@dataclass
class RuntimeContext:
    df: pd.DataFrame

In [4]:
# Cell 3: Fixed execute_code with better variable handling
from langchain_core.tools import tool
from langgraph.runtime import get_runtime
import sys
from io import StringIO
import inspect

@tool
def execute_code(code: str) -> str:
    """Execute Python code blocks including plots, models, and multi-line operations.
    
    Args:
        code: Python code block. Use 'df' for the DataFrame, 'pd' for pandas,
              'plt' for matplotlib, 'np' for numpy.
    """
    runtime = get_runtime(RuntimeContext)
    df = runtime.context.df
    
    # Capture stdout
    old_stdout = sys.stdout
    sys.stdout = captured_output = StringIO()
    
    # Prepare namespace with common libraries
    namespace = {
        'df': df,
        'pd': pd,
        'plt': plt,
        'np': __import__('numpy'),
    }
    
    # Try to import optional libraries
    try:
        import seaborn as sns
        namespace['sns'] = sns
    except ImportError:
        pass
    
    try:
        # Execute the code
        exec(code, namespace)
        
        # Get printed output
        output = captured_output.getvalue()
        
        # Check if a plot was created
        plot_created = False
        if plt.get_fignums():
            plot_created = True
            buf = io.BytesIO()
            plt.savefig(buf, format='png', bbox_inches='tight')
            plt.close('all')
            buf.seek(0)
            img_base64 = base64.b64encode(buf.read()).decode()
        
        # Collect interesting variables from namespace
        result_vars = {}
        excluded_names = {'df', 'pd', 'plt', 'np', 'sns'}
        
        for key, value in namespace.items():
            # Skip private variables and excluded names
            if key.startswith('_') or key in excluded_names:
                continue
            
            # Skip functions and classes (but keep instances)
            if inspect.isfunction(value) or inspect.isclass(value):
                continue
            
            # Skip modules
            if inspect.ismodule(value):
                continue
            
            # Include everything else (numbers, strings, sklearn models, numpy arrays, etc.)
            try:
                # Try to convert to string (some objects can't be stringified)
                str_value = str(value)
                # Limit length to avoid huge outputs
                if len(str_value) > 500:
                    str_value = str_value[:500] + "..."
                result_vars[key] = str_value
            except:
                result_vars[key] = f"<{type(value).__name__} object>"
        
        # Build output
        output_parts = []
        
        if output:
            output_parts.append(f"Output:\n{output.strip()}")
        
        if result_vars:
            vars_str = "\n".join([f"{k} = {v}" for k, v in result_vars.items()])
            output_parts.append(f"Variables created:\n{vars_str}")
        
        if plot_created:
            output_parts.append("[Plot created and displayed]")
        
        if output_parts:
            return "\n\n".join(output_parts)
        else:
            return "Code executed successfully (no output)"
        
    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        return f"Error: {str(e)}\n\nDetails:\n{error_details}"
    finally:
        sys.stdout = old_stdout

In [5]:
@tool
def query_dataframe(pandas_code: str) -> str:
    """Quick pandas operations (single expressions only).
    
    Args:
        pandas_code: Single pandas expression (e.g., "df.head()", "df.describe()")
    """
    runtime = get_runtime(RuntimeContext)
    df = runtime.context.df

    try:
        result = eval(pandas_code, {"df": df, "pd": pd})
        return str(result)
    except Exception as e:
        return f"Error: {e}"

In [6]:
SYSTEM_PROMPT = """You are a data analyst and data scientist assistant.

Tools available:
1. query_dataframe: For simple, single-line pandas operations
   - Use for: df.head(), df.describe(), df.shape, etc.

2. execute_code: For complex operations, visualizations, and models
   - Use for: plots, multi-line code, model training, data transformations
   - Available imports: pd (pandas), plt (matplotlib), np (numpy)
   - The DataFrame is available as 'df'

Rules:
- For simple queries, use query_dataframe
- For plots or complex analysis, use execute_code
- When creating plots, use plt.figure(), create the plot, then plt.show()
- **IMPORTANT: Always use print() to display results like metrics, scores, statistics**
- When calculating metrics (precision, recall, accuracy, etc.), ALWAYS print them
- If code fails, debug and try again

Examples:
- Simple: query_dataframe("df.shape")
- Plot: execute_code("plt.figure()\\nplt.hist(df['Age'])\\nplt.title('Age Distribution')\\nplt.show()")
- Metrics: execute_code("from sklearn.metrics import accuracy_score\\nacc = accuracy_score(y_test, y_pred)\\nprint(f'Accuracy: {acc}')")
"""

In [7]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

df = pd.read_csv("titanic.csv")

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[query_dataframe, execute_code],  # Both tools available
    system_prompt=SYSTEM_PROMPT,
    checkpointer=InMemorySaver(),
    context_schema=RuntimeContext,
)

In [8]:
# Cell: Plot display helper
from IPython.display import Image, display

def run_with_plots(question: str, thread_id: str = "1"):
    """Run agent and display any plots created."""
    config = {"configurable": {"thread_id": thread_id}}
    
    for step in agent.stream(
        {"messages": question},
        context=RuntimeContext(df=df),
        config=config,
        stream_mode="values",
    ):
        last_msg = step["messages"][-1]
        last_msg.pretty_print()
        
        # Check if this was a tool response with a plot
        if hasattr(last_msg, 'content') and '[Plot saved' in str(last_msg.content):
            # The plot was created, matplotlib should show it
            plt.show()

#### Test 1: Simple query
run_with_plots("What's the average age?", "test1")

#### Test 2: Visualization
run_with_plots("Show me a bar chart of survival by passenger class", "test2")

#### Test 3: Statistical analysis
run_with_plots("Calculate correlation between Age and Fare, show as heatmap", "test3")

#### Test 4: Machine learning
run_with_plots("Train a decision tree to predict survival and show feature importance", "test4")

#### Test 5: Complex transformation
run_with_plots("""
Create a new feature 'FamilySize' (SibSp + Parch + 1).
Then create a violin plot showing Fare distribution by FamilySize.
""", "test5")

In [9]:
run_with_plots("Apply the necessary preprocessing steps on the data. Then train a logistic regression model on this dataset. Then, give me the precision and recall scores of the trained model.")

================================ Human Message =================================

Apply the necessary preprocessing steps on the data. Then train a logistic regression model on this dataset. Then, give me the precision and recall scores of the trained model.
================================== Ai Message ==================================

To start, I will first analyze the dataset to determine the necessary preprocessing steps. This may include handling missing values, encoding categorical variables, and normalizing or scaling the data if needed. 

Let's first take a look at the dataset to understand its structure and any preprocessing requirements. I will start with checking the first few rows and the data types. 

Let's proceed with that.
Tool Calls:
  query_dataframe (call_yhqD4zA52tf7sTJ74r5CUuJ1)
 Call ID: call_yhqD4zA52tf7sTJ74r5CUuJ1
  Args:
    pandas_code: df.head()
  query_dataframe (call_PcUbgIDjUNdnCPKTT4KjUuQ8)
 Call ID: call_PcUbgIDjUNdnCPKTT4KjUuQ8
  Args:
    pandas_cod

<string>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


<string>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform

================================= Tool Message =================================
Name: execute_code

Output:
Precision: 0.7857142857142857
Recall: 0.7432432432432432

Variables created:
le = LabelEncoder()
features =      Pclass  Sex       Age  SibSp  Parch      Fare  Embarked
0         3    1 -0.565736      1      0 -0.502445         2
1         1    0  0.663861      1      0  0.786845         0
2         3    0 -0.258337      0      0 -0.488854         2
3         1    0  0.433312      1      0  0.420730         2
4         3    1  0.433312      0      0 -0.486337         2
..      ...  ...       ...    ...    ...       ...       ...
886       2    1 -0.181487      0      0 -0.386671         2
887       1 ...
labels = 0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64
scaler = StandardScaler()
X_train =      Pclass  Sex       Age  SibSp  Parch      Fare  Embarked
331       1    1  1.240235      